Run MLP (first layer weights fixed) on mnist and compute bias and variance

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import queue
import os
import sys
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import copy 
import pandas as pd
class MLP(nn.Module):
    def __init__(self, p, d, o):
        """RF_models
        
        Args:
            p (int): the hidden size
            d (int): the input feature dimension
            o (int): the output dimension
        """
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(d, p, bias=False)
        self.fc2 = nn.Linear(p, o, bias=False)
        self.p = p
        self.d = d
        self.o = o 
    def forward(self, x):
        o = F.relu(self.fc1(x))
        o = self.fc2(o)
        return o
class Ensemble_Two_Layer_NN(object):
    def __init__(self, n_classifiers, p, d=784, o=10):
        """Ensemble_Two_Layer_NN
        
        Args:
            p (int): the hidden size
            d (int, optional): the input feature dimension
            o (int, optional): the output dimension
            coef (float, optional): the ridge regression penalty coefficient
        """
        self.n_classifiers = n_classifiers
        self.p = p
        self.d = d 
        self.o = o 
        self.coef = coef
        self.learners = queue.LifoQueue(maxsize = self.n_classifiers)
        self.MODEL_TYPE = MLP
    def __len__(self):
        return len(self.learners.queue)
    
    def put_model_rho(self, model, rho):
        self.learners.put([model, rho])
    def get_init_model(self, cuda=True):
        model = self.MODEL_TYPE(self.p, self.d, self.o)
        if cuda:
            model.cuda()
        return model
    def cuda(self):
        if len(self) == 0:
            return 
        else:
            for model, rho in self.learners.queue:
                model.cuda()
            return
    def train(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.train()
    def eval(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.eval()
    def forward(self, x):
        Bs = x.size(0)
        if len(self) == 0:
            zeros = torch.zeros(Bs, self.o)
            zeros = zeros.to(x.device)
            return zeros
        else:
            outputs = torch.zeros(Bs, self.o)
            outputs = outputs.to(x.device) 
            for model, rho in self.learners.queue:
                output = model(x)
                outputs += rho*output
            return outputs


In [2]:
def get_subsample_dataset(trainset, subset):
    trainsubset = copy.deepcopy(trainset)
    trainsubset.data = [trainsubset.data[index] for index in subset]
    trainsubset.targets = [trainsubset.targets[index] for index in subset]
    return trainsubset
def fix_width_number(width, n_classifiers):
    return max(1, width//n_classifiers)

# Training
def train(net, trainset, permute_index, train_size, num_iters, lr, batch_size, coef):
    net.train()
    subsample_indexes = np.random.choice(permute_index, size=train_size)
    trainsubset = get_subsample_dataset(trainset, subsample_indexes)
    trainloader = torch.utils.data.DataLoader(trainsubset, batch_size=batch_size, shuffle=True)

    for i_c in range(net.n_classifiers):
        i_iter = 0
        model = net.get_init_model(cuda=True)
        rho = 1/net.n_classifiers
        optimizer = torch.optim.SGD(model.fc2.parameters(), lr=lr)
        #lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = num_iters//3, gamma = 0.1)
        while i_iter < num_iters:
            for inputs, targets in trainloader:
                Bs = inputs.size(0)
                inputs = inputs.reshape(Bs, -1)
                inputs, targets = inputs.cuda(), targets.cuda()
                targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
                targets_onehot.zero_()
                targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets_onehot)
                l2_reg = 0
                for param in model.fc2.parameters():
                    l2_reg += coef * torch.norm(param)
                mse_loss = loss.item()
                l2_loss = l2_reg.item()
                loss += l2_reg
                loss.backward()
                optimizer.step()
                #lr_scheduler.step()
                string = "Train {} model: Iters [{}/{}] mse: {:.4f}, l2_loss: {:.4f}, train_loss:{:.4f}".format(i_c+1, i_iter, num_iters,  mse_loss, l2_loss, loss.item())
                sys.stdout.write(string+"\r")
                sys.stdout.flush()
                i_iter +=1
        net.put_model_rho(model, rho)
    # after all models were trained, estimate the mse error
    train_loss = 0
    correct = 0
    total = 0
    for inputs, targets in trainloader:
        Bs = inputs.size(0)
        inputs = inputs.reshape(Bs, -1)
        inputs, targets = inputs.cuda(), targets.cuda()
        targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
        targets_onehot.zero_()
        targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
        outputs = net.forward(inputs)
        loss = criterion(outputs, targets_onehot)
        train_loss = loss.item() * outputs.numel()
        _, predicted = outputs.max(1)
        correct = predicted.eq(targets).sum().item()
        total = targets.size(0)
    return train_loss/ total , 100. * correct / total

# Test
def test(net, testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            loss = criterion(outputs, targets_onehot)
            test_loss += loss.item() * outputs.numel()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
    return test_loss / total, 100. * correct / total

def compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED):
    net.eval()
    bias2 = 0
    variance = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            OUTPUST_SUM[total:(total + targets.size(0)), :] += outputs
            OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)] += outputs.norm(dim=1) ** 2.0

            bias2 += (OUTPUST_SUM[total:total + targets.size(0), :] / (trial + 1) - targets_onehot).norm() ** 2.0
            variance += OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)].sum()/(trial + 1) - (OUTPUST_SUM[total:total + targets.size(0), :]/(trial + 1)).norm() ** 2.0
            total += targets.size(0)

    return bias2 / total, variance / total


In [3]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)
# loss definition
criterion = nn.MSELoss(reduction='mean').cuda()

def run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, save_csv, num_iters, lr, batch_size_list, K = 1):
    df = pd.DataFrame()
    for batch_size in batch_size_list:
        for train_size in train_sizes:
            hidden_sizes = P_Ns * train_size
            hidden_sizes = np.unique([int(np.around(x)) for x in hidden_sizes])
            for hidden_size in hidden_sizes:
                TRAIN_ACC_SUM = 0.0
                TEST_ACC_SUM = 0.0
                TRAIN_LOSS_SUM = 0.0
                TEST_LOSS_SUM = 0.0
                permute_index = np.random.permutation(len(trainset))
                OUTPUST_SUM = torch.Tensor(test_size, num_classes).zero_().cuda()
                OUTPUTS_SUMNORMSQUARED = torch.Tensor(test_size).zero_().cuda()
                for trial in range(num_trials):
                    net = Ensemble_Two_Layer_NN(n_classifiers = K, p = fix_width_number(hidden_size, K), d=feature_dim, o=num_classes)
                    net.cuda()
                    train_loss, train_acc = train(net, trainset, permute_index, train_size,
                                                 num_iters, lr, batch_size, coef)
                    test_loss, test_acc = test(net, testloader)

                    TRAIN_LOSS_SUM += train_loss
                    TEST_LOSS_SUM += test_loss
                    TRAIN_ACC_SUM += train_acc
                    TEST_ACC_SUM += test_acc

                    # compute bias and variance
                    bias2, variance = compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED)
                    variance_unbias = variance * num_trials / (num_trials - 1.0)
                    bias2_unbias = TEST_LOSS_SUM / (trial + 1) - variance_unbias
                    print('Train size: [{}] hidden size: [{}] batch size: [{}] trial: {}, train_loss: {:.6f}, train acc: {}, test loss: {:.6f}, test acc: {}, bias2: {}, variance: {}'.format(
                        train_size, hidden_size, batch_size,
                        trial, TRAIN_LOSS_SUM / (trial + 1), TRAIN_ACC_SUM / (trial + 1), TEST_LOSS_SUM / (trial + 1),
                        TEST_ACC_SUM / (trial + 1), bias2_unbias, variance_unbias))
                    torch.cuda.empty_cache()
                print('#'*50)
                df = df.append({'train_size': train_size, 'hidden_size':hidden_size, 'batch_size': batch_size,
                                'train_loss': TRAIN_LOSS_SUM / (trial + 1), 'train_acc': TRAIN_ACC_SUM / (trial + 1),
                                'test_loss': TEST_LOSS_SUM / (trial + 1), 'test_acc': TEST_ACC_SUM / (trial + 1), 
                               'variance': variance_unbias.item(),
                               'bias2': bias2_unbias.item()}, ignore_index=True)
                df.to_csv(os.path.join(outdir, save_csv))
    df.to_csv(os.path.join(outdir, save_csv))

In [ ]:
num_classes = 10
num_trials = 50
coef = 0.001
N_Ds = [1]
feature_dim = 784
lr = 0.01 
batch_size_list = [10, 784]
train_sizes = [int(np.around(x*feature_dim)) for x in N_Ds]
test_size = 10000
P_Ns = 10** np.linspace(-2, 1, 50)

for num_iters in [500, 5000]:
    outdir = 'mnist_SGD/num_iters_{}_coef={}'.format(num_iters, coef)
    print(outdir)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, 'singleNN_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 1)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
                 outdir, 'ensembleNNK=2_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 2)

mnist_SGD/num_iters_500_coef=0.001
Train size: [784] hidden size: [8] batch size: [10] trial: 0, train_loss: 1.204033, train acc: 0.0, test loss: 1.120322, test acc: 8.62, bias2: 1.1203222274780273, variance: -3.8925479506346505e-10
Train size: [784] hidden size: [8] batch size: [10] trial: 1, train_loss: 1.045916, train acc: 12.5, test loss: 1.061827, test acc: 10.95, bias2: 0.978081464767456, variance: 0.08374582976102829
Train size: [784] hidden size: [8] batch size: [10] trial: 2, train_loss: 1.080925, train acc: 8.333333333333334, test loss: 1.069227, test acc: 12.196666666666665, bias2: 0.9447241425514221, variance: 0.12450306862592697
Train size: [784] hidden size: [8] batch size: [10] trial: 3, train_loss: 1.076859, train acc: 12.5, test loss: 1.078112, test acc: 11.542499999999999, bias2: 0.9364995956420898, variance: 0.1416119635105133
Train size: [784] hidden size: [8] batch size: [10] trial: 4, train_loss: 1.124586, train acc: 10.0, test loss: 1.097511, test acc: 11.5639999

Train size: [784] hidden size: [8] batch size: [10] trial: 38, train_loss: 1.132124, train acc: 14.102564102564102, test loss: 1.132697, test acc: 10.90076923076923, bias2: 0.9025207757949829, variance: 0.2301764190196991
Train size: [784] hidden size: [8] batch size: [10] trial: 39, train_loss: 1.134295, train acc: 13.75, test loss: 1.132459, test acc: 10.96275, bias2: 0.9016899466514587, variance: 0.2307688444852829
Train size: [784] hidden size: [8] batch size: [10] trial: 40, train_loss: 1.132978, train acc: 13.414634146341463, test loss: 1.132258, test acc: 10.879999999999999, bias2: 0.9017515182495117, variance: 0.23050665855407715
Train size: [784] hidden size: [8] batch size: [10] trial: 41, train_loss: 1.132569, train acc: 13.095238095238095, test loss: 1.132137, test acc: 11.003095238095238, bias2: 0.9021018743515015, variance: 0.23003509640693665
Train size: [784] hidden size: [8] batch size: [10] trial: 42, train_loss: 1.135850, train acc: 12.790697674418604, test loss: 1.1

Train size: [784] hidden size: [9] batch size: [10] trial: 26, train_loss: 1.141658, train acc: 11.11111111111111, test loss: 1.141780, test acc: 11.015925925925924, bias2: 0.9157178401947021, variance: 0.22606220841407776
Train size: [784] hidden size: [9] batch size: [10] trial: 27, train_loss: 1.132208, train acc: 11.607142857142858, test loss: 1.141164, test acc: 10.982142857142856, bias2: 0.9160647392272949, variance: 0.225099578499794
Train size: [784] hidden size: [9] batch size: [10] trial: 28, train_loss: 1.132772, train acc: 12.068965517241379, test loss: 1.142178, test acc: 10.932413793103446, bias2: 0.9162260890007019, variance: 0.22595246136188507
Train size: [784] hidden size: [9] batch size: [10] trial: 29, train_loss: 1.128860, train acc: 12.5, test loss: 1.139393, test acc: 11.196333333333333, bias2: 0.9136346578598022, variance: 0.22575844824314117
Train size: [784] hidden size: [9] batch size: [10] trial: 30, train_loss: 1.132430, train acc: 12.903225806451612, test 

Train size: [784] hidden size: [10] batch size: [10] trial: 14, train_loss: 1.058351, train acc: 15.0, test loss: 1.114067, test acc: 11.677333333333333, bias2: 0.8914790749549866, variance: 0.22258810698986053
Train size: [784] hidden size: [10] batch size: [10] trial: 15, train_loss: 1.052210, train acc: 14.0625, test loss: 1.112781, test acc: 11.521875, bias2: 0.8893368244171143, variance: 0.2234443724155426
Train size: [784] hidden size: [10] batch size: [10] trial: 16, train_loss: 1.058893, train acc: 14.705882352941176, test loss: 1.112560, test acc: 11.375882352941176, bias2: 0.8894376158714294, variance: 0.22312255203723907
Train size: [784] hidden size: [10] batch size: [10] trial: 17, train_loss: 1.052693, train acc: 15.277777777777779, test loss: 1.107952, test acc: 11.764444444444443, bias2: 0.8862380981445312, variance: 0.22171422839164734
Train size: [784] hidden size: [10] batch size: [10] trial: 18, train_loss: 1.063205, train acc: 14.473684210526315, test loss: 1.10989

Train size: [784] hidden size: [12] batch size: [10] trial: 1, train_loss: 1.083749, train acc: 12.5, test loss: 1.157697, test acc: 12.36, bias2: 1.0130289793014526, variance: 0.14466798305511475
Train size: [784] hidden size: [12] batch size: [10] trial: 2, train_loss: 1.082794, train acc: 8.333333333333334, test loss: 1.140960, test acc: 12.42, bias2: 0.9670823216438293, variance: 0.1738772839307785
Train size: [784] hidden size: [12] batch size: [10] trial: 3, train_loss: 1.128002, train acc: 6.25, test loss: 1.135188, test acc: 13.4525, bias2: 0.9420695304870605, variance: 0.19311869144439697
Train size: [784] hidden size: [12] batch size: [10] trial: 4, train_loss: 1.142591, train acc: 10.0, test loss: 1.143471, test acc: 13.16, bias2: 0.9401010274887085, variance: 0.20336973667144775
Train size: [784] hidden size: [12] batch size: [10] trial: 5, train_loss: 1.145640, train acc: 8.333333333333334, test loss: 1.141009, test acc: 12.448333333333332, bias2: 0.936216413974762, varian

...

Train size: [784] hidden size: [468] batch size: [784] trial: 39, train_loss: 0.581321, train acc: 72.63392857142854, test loss: 0.621529, test acc: 68.07674999999998, bias2: 0.49009621143341064, variance: 0.1314324289560318
Train size: [784] hidden size: [468] batch size: [784] trial: 40, train_loss: 0.581413, train acc: 72.62319561971127, test loss: 0.621192, test acc: 68.12682926829267, bias2: 0.48983752727508545, variance: 0.1313544064760208
Train size: [784] hidden size: [468] batch size: [784] trial: 41, train_loss: 0.581541, train acc: 72.64941690962097, test loss: 0.621357, test acc: 68.15690476190474, bias2: 0.48977476358413696, variance: 0.13158206641674042
Train size: [784] hidden size: [468] batch size: [784] trial: 42, train_loss: 0.581246, train acc: 72.65365448504981, test loss: 0.621090, test acc: 68.19534883720928, bias2: 0.4894142150878906, variance: 0.13167595863342285
Train size: [784] hidden size: [468] batch size: [784] trial: 43, train_loss: 0.581671, train acc: 

Train size: [784] hidden size: [538] batch size: [784] trial: 26, train_loss: 0.561294, train acc: 74.14021164021165, test loss: 0.602348, test acc: 70.03444444444443, bias2: 0.47577399015426636, variance: 0.12657387554645538
Train size: [784] hidden size: [538] batch size: [784] trial: 27, train_loss: 0.561562, train acc: 74.18002915451895, test loss: 0.602409, test acc: 70.05107142857142, bias2: 0.4756424129009247, variance: 0.1267661154270172
Train size: [784] hidden size: [538] batch size: [784] trial: 28, train_loss: 0.561266, train acc: 74.30066854327939, test loss: 0.602513, test acc: 70.08620689655172, bias2: 0.4755009114742279, variance: 0.12701252102851868
Train size: [784] hidden size: [538] batch size: [784] trial: 29, train_loss: 0.561147, train acc: 74.29421768707483, test loss: 0.602715, test acc: 70.03466666666667, bias2: 0.47552764415740967, variance: 0.12718747556209564
Train size: [784] hidden size: [538] batch size: [784] trial: 30, train_loss: 0.561626, train acc: 

Train size: [784] hidden size: [620] batch size: [784] trial: 13, train_loss: 0.538960, train acc: 76.77660349854229, test loss: 0.584546, test acc: 71.73357142857142, bias2: 0.465985506772995, variance: 0.11856047064065933
Train size: [784] hidden size: [620] batch size: [784] trial: 14, train_loss: 0.539382, train acc: 76.60714285714288, test loss: 0.584508, test acc: 71.70333333333333, bias2: 0.46520334482192993, variance: 0.11930496245622635
Train size: [784] hidden size: [620] batch size: [784] trial: 15, train_loss: 0.539070, train acc: 76.63424744897961, test loss: 0.584393, test acc: 71.809375, bias2: 0.4648485481739044, variance: 0.11954466253519058
Train size: [784] hidden size: [620] batch size: [784] trial: 16, train_loss: 0.539471, train acc: 76.62815126050421, test loss: 0.585162, test acc: 71.7535294117647, bias2: 0.4649440348148346, variance: 0.12021789699792862
Train size: [784] hidden size: [620] batch size: [784] trial: 17, train_loss: 0.539833, train acc: 76.5943877

Train size: [784] hidden size: [714] batch size: [784] trial: 0, train_loss: 0.511490, train acc: 79.71938775510205, test loss: 0.564658, test acc: 73.19, bias2: 0.5646582245826721, variance: 1.0120625226761604e-08
Train size: [784] hidden size: [714] batch size: [784] trial: 1, train_loss: 0.515350, train acc: 79.71938775510205, test loss: 0.563078, test acc: 74.33500000000001, bias2: 0.5037022829055786, variance: 0.059375397861003876
Train size: [784] hidden size: [714] batch size: [784] trial: 2, train_loss: 0.511638, train acc: 79.29421768707483, test loss: 0.563408, test acc: 73.76666666666667, bias2: 0.4822056293487549, variance: 0.08120274543762207
Train size: [784] hidden size: [714] batch size: [784] trial: 3, train_loss: 0.513393, train acc: 79.55994897959184, test loss: 0.564535, test acc: 74.0275, bias2: 0.4714549779891968, variance: 0.09307967126369476
Train size: [784] hidden size: [714] batch size: [784] trial: 4, train_loss: 0.513481, train acc: 78.59693877551021, test 

Train size: [784] hidden size: [714] batch size: [784] trial: 37, train_loss: 0.513630, train acc: 79.10848549946292, test loss: 0.565199, test acc: 73.70631578947366, bias2: 0.44554635882377625, variance: 0.11965300887823105
Train size: [784] hidden size: [714] batch size: [784] trial: 38, train_loss: 0.514026, train acc: 79.01622187336471, test loss: 0.565250, test acc: 73.71358974358972, bias2: 0.4455745816230774, variance: 0.11967581510543823
Train size: [784] hidden size: [714] batch size: [784] trial: 39, train_loss: 0.514001, train acc: 79.04017857142856, test loss: 0.564925, test acc: 73.74924999999998, bias2: 0.4453010857105255, variance: 0.11962398141622543
Train size: [784] hidden size: [714] batch size: [784] trial: 40, train_loss: 0.513966, train acc: 79.05052264808361, test loss: 0.564705, test acc: 73.77634146341461, bias2: 0.44502073526382446, variance: 0.11968434602022171
Train size: [784] hidden size: [714] batch size: [784] trial: 41, train_loss: 0.513903, train acc:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
font = {
        'size'   : 18}
matplotlib.rc('font', **font)
figsize = (16, 5)
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd

def plot_bias_var(df, N_D, ymin=0, ymax=1.0):
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    axes1[0].set_xscale('log')
    axes1[1].set_xscale('log')
    axes1[2].set_xscale('log')
    cur_df = df[df['train_size']/feature_dim==N_D]
    test_loss = cur_df['test_loss']
    bias2 = cur_df['bias2']
    var = cur_df['variance']
    P_N = cur_df['hidden_size']/cur_df['train_size']
    axes1[0].plot(P_N, test_loss)
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    axes1[1].plot(P_N, bias2)
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    axes1[2].plot(P_N, var)
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.show()
def plot_single_vs_ensemble(dfs_list, Ks_list, N_D, feature_dim, ymin=0, ymax=1.0):
    assert len(dfs_list) == len(Ks_list)
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    for i in range(3):
        axes1[i].set_xscale('log')
    dfs_list = [df[df['train_size']/feature_dim==N_D] for df in dfs_list]
    for cur_df, K in zip(dfs_list, Ks_list):
        test_loss = cur_df['test_loss']
        bias2 = cur_df['bias2']
        var = cur_df['variance']
        P_N = cur_df['hidden_size']/cur_df['train_size']
        axes1[0].plot(P_N, test_loss, label='K={}'.format(K))
        axes1[1].plot(P_N, bias2, label='K={}'.format(K))
        axes1[2].plot(P_N, var, label='K={}'.format(K))
    
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.legend()
    plt.show()


In [ ]:
# K2_df = pd.read_csv(os.path.join(outdir, 'ensembleNNK=2_output.csv'))
# K1_df = pd.read_csv(os.path.join(outdir, 'singleNN_output.csv'))
# plot_single_vs_ensemble([K1_df, K2_df], [1, 2], N_Ds[0], 784,)